In [20]:
import pandas as pd
import geopandas as gpd

from utils.constants import Paths

In [21]:
rating_df = pd.read_csv("/Users/DonGuest/Downloads/dot_V_SSS_SEGMENTRATING_1_20190129.csv")
main_df = pd.read_parquet(Paths.PROCESSED_DATA_PARQUET / "aggregated_pothole.parquet")

In [22]:
from shapely import wkt

In [23]:
rating_df['the_geom'] = rating_df['the_geom'].apply(wkt.loads)
rating_df.rename(columns={'the_geom': 'geometry'}, inplace=True)

In [24]:
rating_gdf = gpd.GeoDataFrame(rating_df, geometry='geometry')

In [25]:
rating_gdf.crs = {'init': 'epsg:4326'}

/Users/DonGuest/Code/simcity-smart-cities/simcity_env/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [26]:
main_df['geometry'] = main_df['geometry'].apply(wkt.loads)

In [27]:
rating_gdf['RatingDate'] = pd.to_datetime(rating_gdf['RatingDate'], format='%m/%d/%Y %I:%M:%S %p')

In [28]:
main_gdf = gpd.GeoDataFrame(main_df, geometry='geometry')
main_gdf.crs = {'init': 'epsg:4326'}

/Users/DonGuest/Code/simcity-smart-cities/simcity_env/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [29]:
rating_gdf = rating_gdf[['geometry', 'RatingDate', 'Rating_B']]

In [30]:
joined_gdf = gpd.sjoin_nearest(main_gdf, rating_gdf, how='inner')

/Users/DonGuest/Code/simcity-smart-cities/simcity_env/lib/python3.11/site-packages/geopandas/array.py:365: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [31]:
joined_gdf.dropna(subset=['Rating_B', 'RatingDate'], inplace=True)

In [32]:
joined_gdf['time_difference'] = (joined_gdf['date_created'] - joined_gdf['RatingDate']).abs()

max_time_diff = pd.Timedelta(days=300)
filtered_gdf = joined_gdf[joined_gdf['time_difference'] <= max_time_diff]

filtered_gdf['point_id'] = range(len(filtered_gdf))
idx = filtered_gdf.groupby('point_id')['time_difference'].idxmin()
final_join = filtered_gdf.loc[idx]

/Users/DonGuest/Code/simcity-smart-cities/simcity_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [33]:
joined_gdf['RatingDate'] = pd.to_datetime(joined_gdf['RatingDate'], format='%m/%d/%Y %I:%M:%S %p')